Deriving z.

In [2]:
# modifying "reading in financial data" to 32 assets and right T value

import numpy as np
import pandas as pd

# You may choose to select different parameters/values
number_assets = 32
T  = 392
# Read returns
df = pd.read_csv('returns_data.txt', sep='\s+')

Rraw = df.values.T

# Select the first N,T assets and scenarios, you may choose a different strategy if you would like to do so.
R = Rraw[:number_assets,:T]

# Expected return of each asset
expected_returns = np.mean(R, axis = 1)

# Covariance matrix of asset returns
covariance_matrix = np.cov(R)

mean_returns = df.mean()
std_dev_returns = df.std()

# Set the confidence level
confidence_level = 0.95

# Calculate the z-score for the confidence level
from scipy.stats import norm
z_score = norm.ppf(confidence_level)

# Calculate VaR at the 95% confidence level
VaR_95 = mean_returns - z_score * std_dev_returns

print(f"VaR at 95% confidence level: {VaR_95}")
print(f"Z-score for 95% confidence level: {z_score}")

/tmp/ipykernel_219/316017550.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


VaR at 95% confidence level: -1.9863165e-02   -0.054498
-1.9305019e-02   -0.052114
4.3494020e-03    -0.065133
-6.5832785e-03   -0.073864
1.1202068e-02    -0.051338
-1.6676961e-02   -0.080386
-3.1958164e-03   -0.062213
-3.6061820e-02   -0.086725
-6.2434964e-03   -0.068405
5.3885423e-02    -0.041255
1.9841270e-02    -0.053425
5.2570093e-03    -0.048105
-8.8888889e-03   -0.074956
-4.9275362e-03   -0.045472
-2.0058504e-02   -0.061249
3.0338181e-03    -0.103650
-2.5770824e-02   -0.057937
0.0000000e+00    -0.079655
-1.7283951e-02   -0.096552
1.1389522e-02    -0.078960
-1.2942779e-02   -0.049639
-1.5117830e-02   -0.051520
8.3017112e-04    -0.040895
4.6242775e-03    -0.058778
-2.3952096e-03   -0.069087
2.4055629e-02    -0.077388
-1.7333333e-02   -0.073769
-6.4360418e-03   -0.047025
1.0585888e-02    -0.116658
3.3211333e-02    -0.064501
-2.9184038e-02   -0.035317
-9.4182825e-03   -0.045576
dtype: float64
Z-score for 95% confidence level: 1.6448536269514722


In [3]:
# import random weight function from random_weights.ipynb

def generate_random_weights(number_of_assets):
    weights = np.random.rand(number_of_assets) # Generate random numbers
    weights /= np.sum(weights) # Normalize so that the sum is 1
    return weights

# Test the function with 32 assets
n_assets = 32
random_weights = generate_random_weights(n_assets)
random_weights, sum(random_weights)


(array([0.00418642, 0.02575848, 0.06693708, 0.06799167, 0.05898317,
        0.07067119, 0.02501272, 0.01295003, 0.04382892, 0.06962844,
        0.05880262, 0.02637454, 0.02220247, 0.01063647, 0.01108875,
        0.00419621, 0.00399815, 0.0395862 , 0.02611385, 0.06111845,
        0.07534739, 0.01170318, 0.02671506, 0.01058358, 0.04007832,
        0.00407736, 0.0111234 , 0.01482553, 0.02636917, 0.03315742,
        0.02972971, 0.00622408]),
 1.0)

In [4]:
# Redefining the calculate_historical_VaR function
def calculate_historical_VaR(weights, mu_R, confidence_level=0.95):
    # Calculate portfolio returns for the given weights
    portfolio_returns = np.dot(mu_R, np.transpose(np.array(weights)))
    # Calculate the VaR at the specified confidence level
    VaR = np.percentile(portfolio_returns, 100 * (1 - confidence_level))
    return -VaR

In [5]:
# Define a function to simulate the random generation of weights and calculation of VaR
def simulate_VaR(num_simulations, num_assets, mu_R, confidence_level=0.95):
    VaR_results = []
    weights_list = []

    for _ in range(num_simulations):
        weights = generate_random_weights(num_assets)
        VaR = calculate_historical_VaR(weights, mu_R, confidence_level)
        VaR_results.append(VaR)
        weights_list.append(weights)
    
    return VaR_results, weights_list

In [6]:
# Example usage
num_simulations = 1000  # The number of times we want to run the simulation
num_assets = 32  # Number of assets in the portfolio

# We need the mu_R matrix to run the simulations which should be the historical returns matrix
# For the sake of the example, let's create a dummy mu_R with random values
# Normally, you would use actual historical returns data for your assets here
mu_R = np.random.randn(1000, num_assets)  # 1000 scenarios, 32 assets

# Run the simulation
VaR_results, weights_list = simulate_VaR(num_simulations, num_assets, mu_R)

# Display the first few results to ensure it's working
VaR_results[:5], weights_list[:5]

([0.3128811166148638,
  0.34202650160735865,
  0.35334276930136604,
  0.3410365740045231,
  0.3165141564733053],
 [array([0.0495565 , 0.04090496, 0.04357528, 0.03138587, 0.04548934,
         0.04759537, 0.00425981, 0.04478657, 0.03877712, 0.04574071,
         0.04282936, 0.04846589, 0.04523607, 0.02474504, 0.00173475,
         0.01009745, 0.04951992, 0.00847554, 0.03235129, 0.02737412,
         0.02521216, 0.01083173, 0.03016904, 0.0412418 , 0.03148121,
         0.04565462, 0.00779222, 0.04414392, 0.04107802, 0.00390577,
         0.03467097, 0.00091758]),
  array([0.01403353, 0.00982509, 0.02215693, 0.04888251, 0.03759727,
         0.04673139, 0.03929041, 0.02254295, 0.0167434 , 0.03030957,
         0.03559299, 0.0318543 , 0.02907078, 0.04154272, 0.01493068,
         0.04450683, 0.04399607, 0.00446831, 0.05255986, 0.03136256,
         0.05252859, 0.04756222, 0.03254053, 0.01271068, 0.00609097,
         0.02039448, 0.05474213, 0.04600814, 0.01125323, 0.04347461,
         0.04532531, 0.0

In [7]:
# let's now actually calculate z

# Parameters
number_of_assets = 32
number_of_portfolios = 1000  # The number of random portfolios you want to generate
confidence_level = 0.95

# Assuming mu_R is the matrix of historical returns for the assets
# Replace this with your actual historical returns data
mu_R = expected_returns  # This should be the historical returns data for your assets
# expected retursn --> z = .001
# mu_R = R.T # z = .0045
# z provided from packages == 1.645

# Generate the random weights for the portfolios and calculate the VaR for each
VaRs = []
for _ in range(number_of_portfolios):
    weights = generate_random_weights(number_of_assets)
    VaR = calculate_historical_VaR(weights, mu_R, confidence_level)
    VaRs.append(VaR)

# Calculate z
mean_VaR = np.mean(VaRs)
median_VaR = np.median(VaRs)
z = mean_VaR  # or median_VaR, depending on your specific requirements

print(f"The mean VaR at a {confidence_level*100}% confidence level is: {mean_VaR}")
print(f"The median VaR at a {confidence_level*100}% confidence level is: {median_VaR}")


The mean VaR at a 95.0% confidence level is: -0.001012471817893799
The median VaR at a 95.0% confidence level is: -0.001007872904990408


In [8]:
# Parameters
number_of_assets = 32
number_of_portfolios = 1000  # The number of random portfolios you want to generate
confidence_level = 0.95

# Assuming mu_R is the matrix of historical returns for the assets
# Replace this with your actual historical returns data
# mu_R = expected_returns  # This should be the historical returns data for your assets
# expected retursn --> z = .001
mu_R = R.T # z = .0045
# z provided from packages == 1.645

# Generate the random weights for the portfolios and calculate the VaR for each
VaRs = []
for _ in range(number_of_portfolios):
    weights = generate_random_weights(number_of_assets)
    VaR = calculate_historical_VaR(weights, mu_R, confidence_level)
    VaRs.append(VaR)

# Calculate z
mean_VaR = np.mean(VaRs)
median_VaR = np.median(VaRs)
z = mean_VaR  # or median_VaR, depending on your specific requirements

print(f"The mean VaR at a {confidence_level*100}% confidence level is: {mean_VaR}")
print(f"The median VaR at a {confidence_level*100}% confidence level is: {median_VaR}")

The mean VaR at a 95.0% confidence level is: 0.0455241269789427
The median VaR at a 95.0% confidence level is: 0.045520151841198175
